# Locations info Arrangement

In [1]:
import json
import pandas as pd
import requests

## Hospitals

In [2]:
with open('fangcang-hospitals.json', 'r') as f:
    fangcang = json.loads(f.read())

In [4]:
with open('hospitals-wuhan.json', 'r') as f:
    hospitals = json.loads(f.read())

In [3]:
with open('huoshen-leishen-hospital.json', 'r') as f:
    huoshen_leishen = json.loads(f.read())

In [5]:
locations_df = pd.DataFrame()

Convert fangcang data:

In [6]:
for item in fangcang['data']:
    dfx = pd.DataFrame()
    dfx['type'] = ['Fangcang']
    dfx['title'] = [item['title']]
    dfx['address'] = [item['address']]
    dfx['lat'] = [item['location']['lat']]
    dfx['lng'] = [item['location']['lng']]
    locations_df = locations_df.append(dfx)

Convert hospitals data:

In [7]:
for item in hospitals['data']['data']:
    dfx = pd.DataFrame()
    dfx['type'] = ['Core' if item['isCore']  == 1 else 'Fever']
    dfx['title'] = [item['orgName']]
    dfx['address'] = [item['orgAddr']]
    dfx['lat'] = [item['lat']]
    dfx['lng'] = [item['lng']]
    locations_df = locations_df.append(dfx)

Convert huoshen leishen data:

In [8]:
for item in huoshen_leishen['data']:
    dfx = pd.DataFrame()
    dfx['type'] = ['HuoshenLeishen']
    dfx['title'] = [item['title']]
    dfx['address'] = [item['address']]
    dfx['lat'] = [item['location']['lat']]
    dfx['lng'] = [item['location']['lng']]
    locations_df = locations_df.append(dfx)

In [9]:
locations_df.to_csv('hospitals.csv', index=False)

## Storehouse

In [10]:
with open('store.json', 'r') as f:
    store = json.loads(f.read())

In [11]:
for item in store['data']:
    dfx = pd.DataFrame()
    dfx['type'] = ['Storehouse']
    dfx['title'] = [item['title']]
    dfx['address'] = [item['address']]
    dfx['lat'] = [item['location']['lat']]
    dfx['lng'] = [item['location']['lng']]
    locations_df = locations_df.append(dfx)

In [12]:
locations_df.to_csv('loacations.csv', index=False)

In [13]:
locations_df

,type,title,address,lat,lng
0,Fangcang,方舱医院(新洲区),湖北省武汉市新洲区邾城全民健身中心(永安大道东),30.837821,114.808144
0,Fangcang,方舱医院(白马驿站),湖北省武汉市洪山区欢乐大道,30.598630,114.425040
0,Fangcang,长江新城方舱医院,湖北省武汉市江岸区谌家矶特4号,30.681784,114.366320
0,Fangcang,方舱医院(洪山体育馆),湖北省武汉市武昌区体育馆路特1号,30.545917,114.334634
0,Fangcang,方舱医院(石牌岭高级职业中学),湖北省武汉市洪山区石牌岭路13号,30.524705,114.342449
0,Fangcang,方舱医院(江汉经济开发区),湖北省武汉市江汉区江兴路25号,30.621089,114.228881
0,Fangcang,方舱医院(武汉体育馆),湖北省武汉市硚口区解放大道612号,30.577000,114.260066
0,Fangcang,方舱医院(武汉体育中心),湖北省武汉市蔡甸区车城北路与体育路交叉口北,30.500686,114.171899
0,Fangcang,方舱医院(武钢体育馆),湖北省武汉市洪山区仁和路111号,30.612412,114.388901
0,Fangcang,方舱医院(中国光谷日海),湖北省武汉市江夏区栗庙路6号,30.393470,114.426479


## Get Distances

In [17]:
locations_df[['lat', 'lng']]

,lat,lng
0,30.837821,114.808144
0,30.598630,114.425040
0,30.681784,114.366320
0,30.545917,114.334634
0,30.524705,114.342449
0,30.621089,114.228881
0,30.577000,114.260066
0,30.500686,114.171899
0,30.612412,114.388901
0,30.393470,114.426479


In [27]:
map_key = 'KNHBZ-SNFCD-JCC4P-HAWVD-A2KOO-OOBHO'
url = f'http://apis.map.qq.com/ws/distance/v1/matrix?key={map_key}&mode=driving'

from_str = ''
for index, item in locations_df[locations_df['type'] == 'Storehouse'][['lat', 'lng']].iterrows():
    lat = item['lat']
    lng = item['lng']
    from_str += f'{lat},{lng};'
from_str = from_str[:-1]

to_str = ''
for index, item in locations_df[locations_df['type'] == 'Storehouse'][['lat', 'lng']].iterrows():
    lat = item['lat']
    lng = item['lng']
    to_str += f'{lat},{lng};'
to_str = to_str[:-1]

url = f'{url}&from={from_str}&to={to_str}'

print(url)

http://apis.map.qq.com/ws/distance/v1/matrix?key=KNHBZ-SNFCD-JCC4P-HAWVD-A2KOO-OOBHO&mode=driving&from=30.589509,114.300335;30.506843,114.241888;30.81081,114.226873;30.634988,114.292486;30.55423,114.2185;30.841568,114.807981;30.627385,114.31227;30.57813,114.03364;30.882697,114.377555;30.34257,114.31341;30.50054,114.34295;30.63116,114.39569;30.60574,114.2755;30.65144,114.13959;30.55385,114.315679;30.58229,114.21491;30.50574,114.167526&to=30.589509,114.300335;30.506843,114.241888;30.81081,114.226873;30.634988,114.292486;30.55423,114.2185;30.841568,114.807981;30.627385,114.31227;30.57813,114.03364;30.882697,114.377555;30.34257,114.31341;30.50054,114.34295;30.63116,114.39569;30.60574,114.2755;30.65144,114.13959;30.55385,114.315679;30.58229,114.21491;30.50574,114.167526
